https://www.kaggle.com/asindico/customer-segments-with-pca

In [1]:
import numpy as np 
import pandas as pd 

from subprocess import check_output
print(check_output(["ls", "./input"]).decode("utf8"))

aisles.csv
departments 2.csv
departments.csv
order_products__prior.csv
order_products__train.csv
orders.csv
products.csv
sample_submission.csv



### orders.csv
This file tells to which set (prior, train, test) an order belongs. You are predicting reordered items only for the test set orders. 'order_dow' is the day of week.

In [2]:
orders = pd.read_csv('./input/orders.csv')
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


### order_products__*.csv
These files specify which products were purchased in each order. order_products__prior.csv contains previous order contents for all customers. 'reordered' indicates that the customer has a previous order that contains the product. Note that some orders will have no reordered items.

In [3]:
prior = pd.read_csv('./input/order_products__prior.csv')
prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [4]:
train = pd.read_csv('./input/order_products__train.csv')
train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [5]:
prior = prior[0:30000]

In [6]:
order_prior = pd.merge(prior,orders,on=['order_id','order_id'])
order_prior = order_prior.sort_values(by=['user_id','order_id'])
order_prior.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
14111,1483,32818,1,1,90,prior,31,3,13,5.0
14112,1483,27582,2,1,90,prior,31,3,13,5.0
14113,1483,12302,3,1,90,prior,31,3,13,5.0
14114,1483,1831,4,1,90,prior,31,3,13,5.0
14115,1483,19204,5,1,90,prior,31,3,13,5.0


In [7]:
products = pd.read_csv('./input/products.csv')
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [8]:
aisles = pd.read_csv('./input/aisles.csv')
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [9]:
print(aisles.shape)

(134, 2)


In [10]:
_mt = pd.merge(prior,products, on = ['product_id','product_id'])
_mt = pd.merge(_mt,orders,on=['order_id','order_id'])
mt = pd.merge(_mt,aisles,on=['aisle_id','aisle_id'])
mt.head(10)

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle
0,2,33120,1,1,Organic Egg Whites,86,16,202279,prior,3,5,9,8.0,eggs
1,26,33120,5,0,Organic Egg Whites,86,16,153404,prior,2,0,16,7.0,eggs
2,120,33120,13,0,Organic Egg Whites,86,16,23750,prior,11,6,8,10.0,eggs
3,327,33120,5,1,Organic Egg Whites,86,16,58707,prior,21,6,9,8.0,eggs
4,390,33120,28,1,Organic Egg Whites,86,16,166654,prior,48,0,12,9.0,eggs
5,537,33120,2,1,Organic Egg Whites,86,16,180135,prior,15,2,8,3.0,eggs
6,582,33120,7,1,Organic Egg Whites,86,16,193223,prior,6,2,19,10.0,eggs
7,608,33120,5,1,Organic Egg Whites,86,16,91030,prior,11,3,21,12.0,eggs
8,623,33120,1,1,Organic Egg Whites,86,16,37804,prior,63,3,12,3.0,eggs
9,689,33120,4,1,Organic Egg Whites,86,16,108932,prior,16,1,13,3.0,eggs


In [11]:
mt['product_name'].value_counts()[0:10]

Banana                    464
Bag of Organic Bananas    370
Organic Strawberries      241
Organic Baby Spinach      205
Organic Hass Avocado      193
Organic Avocado           169
Large Lemon               136
Strawberries              133
Organic Raspberries       127
Limes                     120
Name: product_name, dtype: int64

In [12]:
len(mt['product_name'].unique())

8862

In [13]:
len(mt['aisle'].unique())

134

In [14]:
prior.shape

(30000, 4)

In [15]:
len(mt['aisle'].unique())

134

In [16]:
# カウント
mt['aisle'].value_counts()[0:10]

fresh fruits                     3434
fresh vegetables                 3056
packaged vegetables fruits       1597
yogurt                           1360
packaged cheese                   906
milk                              792
water seltzer sparkling water     789
chips pretzels                    624
soy lactosefree                   616
bread                             555
Name: aisle, dtype: int64

In [17]:
# 縦軸と横軸で表を作成
cust_prod = pd.crosstab(mt['user_id'], mt['aisle'])
cust_prod.head(10)

aisle,air fresheners candles,asian foods,baby accessories,baby bath body care,baby food formula,bakery desserts,baking ingredients,baking supplies decor,beauty,beers coolers,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
user_id,,,,,,,,,,,,,,,,,,,,,
90,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
150,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,2,0,0
206,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
382,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
503,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
678,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
787,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
971,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1059,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,3


In [18]:
# before PCA
cust_prod.shape

(2915, 134)

In [22]:
from sklearn.decomposition import PCA
pca = PCA(n_components=6)
pca.fit(cust_prod)
pca_samples = pca.transform(cust_prod)

In [24]:
# after PCA
pca_samples.shape

(2915, 6)

In [23]:
ps = pd.DataFrame(pca_samples)
ps.head()

,0,1,2,3,4,5
0,-1.610934,0.262776,0.170658,-0.453779,-0.208693,3.917336
1,-1.044071,0.453062,-0.247673,0.085537,-0.178395,1.394101
2,0.218872,0.375611,-0.829238,0.156194,-0.544770,-0.076863
3,-1.690545,-0.290570,-0.040737,0.206148,-0.093671,0.347266
4,-0.227766,-0.384007,-0.441972,0.024048,-0.129441,0.098779


In [ ]:
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
tocluster = pd.DataFrame(ps[[4,1]])
print (tocluster.shape)
print (tocluster.head())

fig = plt.figure(figsize=(8,8))
plt.plot(tocluster[4], tocluster[1], 'o', markersize=2, color='blue', alpha=0.5, label='class1')

plt.xlabel('x_values')
plt.ylabel('y_values')
plt.legend()
plt.show()

(2915, 2)
          4         1
0 -0.208693  0.262776
1 -0.178395  0.453062
2 -0.544770  0.375611
3 -0.093671 -0.290570
4 -0.129441 -0.384007


/Users/take/.pyenv/versions/anaconda3-4.1.0/lib/python3.5/site-packages/matplotlib/font_manager.py:1297: UserWarning: findfont: Font family ['IPAmincho'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

clusterer = KMeans(n_clusters=4,random_state=42).fit(tocluster)
centers = clusterer.cluster_centers_
c_preds = clusterer.predict(tocluster)
print(centers)

In [ ]:
print (c_preds[0:100])

In [ ]:
import matplotlib
fig = plt.figure(figsize=(8,8))
colors = ['orange','blue','purple','green']
colored = [colors[k] for k in c_preds]
print (colored[0:10])
plt.scatter(tocluster[4],tocluster[1],  color = colored)
for ci,c in enumerate(centers):
    plt.plot(c[0], c[1], 'o', markersize=8, color='red', alpha=0.9, label=''+str(ci))

plt.xlabel('x_values')
plt.ylabel('y_values')
plt.legend()
plt.show()

In [ ]:
clust_prod = cust_prod.copy()
clust_prod['cluster'] = c_preds

clust_prod.head(10)